In [ ]:
import numpy as np
import random as rd
from random import randint
import time

In [ ]:
class Mochila():

  def __init__(self, nomeArquivo):
    self.itens = []
    self.pesos = []
    self.valores = []

    arquivo = open(nomeArquivo, 'r')

    self.pesoLimite = float(arquivo.readline());
    for linha in arquivo:
      linhaSeparada = linha.split(' ')
      self.itens.append(int(linhaSeparada[0]))
      self.valores.append(float(linhaSeparada[1]))
      self.pesos.append(float(linhaSeparada[2]))
    
  def imprimirMochila(self):
    print('Peso Limite  {}\n'.format(self.pesoLimite))
    print('Itens Disponíveis:')
    for i in range(len(self.itens)):
      print('\nItem  {}'.format(self.itens[i]))
      print('Valor {}'.format(self.valores[i]))
      print('Peso  {}'.format(self.pesos[i]))
    print('--------\n')

  def imprimirMochilaSelecionada(self, itensSelecionados, pesoTotal):
    print('Peso Limite  {}\n'.format(self.pesoLimite))
    print('Peso Total  {}\n'.format(pesoTotal))
    print('Itens Selecionados:')
    for i in range(len(itensSelecionados)):
      if itensSelecionados[i] == 1:
        print('\nItem  {}'.format(self.itens[i]))
        print('Valor {}'.format(self.valores[i]))
        print('Peso  {}'.format(self.pesos[i]))
    print('--------\n')



In [ ]:
class InicializaAlgoritmoGenetico():
  #para que todos os AG's utilizem dos mesmos dados iniciais:
  def __init__(self, mochila):
    self.mochila = mochila

    self.solucoesPorPopulacao = int(len(self.mochila.itens) * 2)

    if self.solucoesPorPopulacao % 2 != 0:
      self.solucoesPorPopulacao = self.solucoesPorPopulacao + 1

    self.tamanhoPopulacao = (self.solucoesPorPopulacao, len(self.mochila.itens))

    solucoes10PorCento = int(self.solucoesPorPopulacao * 0.1)

    self.itensCondicaoParada = 4

    if solucoes10PorCento > 4:
      self.itensCondicaoParada = solucoes10PorCento;

    #inicializa a população randomicamente com 0 ou 1
    self.populacao = np.random.randint(2, size = self.tamanhoPopulacao)
    self.populacao = self.populacao.astype(int)

    self.numeroGeracoes = self.populacao.shape[1] * 6

    self.numeroPais = int(self.tamanhoPopulacao[0]/3)
    if self.numeroPais % 2 != 0:
      self.numeroPais = self.numeroPais - 1

    self.numeroDesc = self.numeroPais 


In [ ]:
class AlgoritmoGenetico():

  def __init__(self, AGInicial, tipoSelecao, tipoCrossover, tipoSelecaoSobreviventes):
    self.mochila = AGInicial.mochila

    self.tipoSelecao = tipoSelecao
    self.tipoCrossover = tipoCrossover
    self.tipoSelecaoSobreviventes = tipoSelecaoSobreviventes

    self.solucoesPorPopulacao = AGInicial.solucoesPorPopulacao
    self.tamanhoPopulacao = AGInicial.tamanhoPopulacao

    self.itensCondicaoParada = AGInicial.itensCondicaoParada

    self.populacao = AGInicial.populacao.copy()

    self.numeroGeracoes = AGInicial.numeroGeracoes
    self.numeroPais = AGInicial.numeroPais
    self.numeroDesc = AGInicial.numeroDesc

  def imprimirPopulacao(self, inicialFinal):
    print('População ' + inicialFinal + ': \n{}\n'.format(self.populacao))

  def imprimirAptidao(self, inicialFinal):
    print('Aptidão da Geração ' + inicialFinal + ': \n{}\n'.format(self.aptidao))

  def imprimirMelhorAptidao(self, inicialFinal):
    print('Melhor Aptidão da Geração ' + inicialFinal + ': \n{}\n'.format(self.getMelhorAptidao()))

  def getMelhorAptidao(self):
    aptidaoMaxima = np.where(self.aptidao == np.max(self.aptidao))
    return "{:.3f}".format(self.aptidao[aptidaoMaxima[0][0]])

  def imprimirPesos(self, inicialFinal):
    print('Pesos da Geração ' + inicialFinal + ': \n{}\n'.format(self.pesoTotal))

  def calculaAptidao(self, populacao):
    self.pesoTotal = np.empty(populacao.shape[0])
    aptidao = np.empty(populacao.shape[0])
    #para cada item da população
    for i in range(populacao.shape[0]):
      #pega a soma dos produtos da população e valor
      somaProdutoPopValor = np.sum(populacao[i] * self.mochila.valores)
      #pega a soma dos produtos da população e peso
      somaProdutoPopPeso = np.sum(populacao[i] * self.mochila.pesos)

      #se o peso for menor ou igual ao limite o fitness guarda esse valor
      if somaProdutoPopPeso <= self.mochila.pesoLimite:
          aptidao[i] = somaProdutoPopValor
      #se a aptidao for inválida, guarda o valor do peso
      else:
          aptidao[i] = -somaProdutoPopPeso
      
      self.pesoTotal[i] = somaProdutoPopPeso

    self.pesoTotal = self.pesoTotal.astype(float)
    aptidao = aptidao.astype(float) 
    return self.transfomarAptidaoNegativa(aptidao)

  def transfomarAptidaoNegativa(self, aptidao):  
    #essa função transforma as aptidões inválidas em válidas com um valor mínimo, 
    #para que o algoritmo seja capaz de selecionar sempre o elemento menos pior da população
    aptidaoOrdenada = sorted(aptidao, reverse=True)
    aptidaoRank = sorted(range(len(aptidao)),key=aptidao.__getitem__, reverse=True)
    for i in range(len(aptidaoOrdenada)):
      if aptidaoOrdenada[i] < 0.0:
        aptidao[aptidaoRank[i]] = (len(aptidaoOrdenada) - i) / 100000
    return aptidao

  def selecao(self):
    aptidaoTotal = np.sum(self.aptidao) 

    if aptidaoTotal == 0:
      self.selecaoAleatoria()
      return

    if self.tipoSelecao == 'Roleta Viciada':
      self.selecaoRoletaViciada()
    elif self.tipoSelecao == 'Torneio':
      self.selecaoTorneio()
    else:
      self.selecaoAleatoria()

  def selecaoRoletaViciada(self):
    self.aptidao = list(self.aptidao)

    self.pais = np.empty((self.numeroPais, self.populacao.shape[1]))

    countPais = 0
    aptidaoAux = np.copy(self.aptidao)  
    while countPais < self.numeroPais:
      for i in range(self.numeroPais):
        aptidaoTotal = np.sum(aptidaoAux) 
        pick = rd.uniform(0, aptidaoTotal)
        current = 0
        for j in range(self.populacao.shape[0]):
          current += aptidaoAux[j]
          if current > pick:
            self.pais[i] = self.populacao[j]
            countPais = countPais + 1
            aptidaoAux[j] = 0
            break;

  def selecaoTorneio(self):
    self.aptidao = list(self.aptidao)

    self.pais = np.empty((self.numeroPais, self.populacao.shape[1]))

    k = int(self.numeroPais/2)
    aptidaoAux = self.aptidao
    for i in range(self.numeroPais):
      aptidaoSample = rd.sample(aptidaoAux, k)
      #seleciona o maior fitness dos pais em k
      aptidaoMaxima = np.where(aptidaoSample == np.max(aptidaoSample)) 
      #seleciona o pai com o maior fitness da lista em k
      self.pais[i,:] = self.populacao[aptidaoMaxima[0][0], :]
      del aptidaoAux[aptidaoMaxima[0][0]]

  def selecaoAleatoria(self):
    self.pais = np.empty((self.numeroPais, self.populacao.shape[1]))
    idx = np.random.randint(self.populacao.shape[0], size=self.pais.shape[0])
    self.pais = self.populacao[idx,:]

  def crossover(self):
    if self.tipoCrossover == 'Um Ponto de Corte':
      self.crossoverUmPonto()
    elif self.tipoCrossover == 'Dois Pontos de Corte':
      self.crossoverDoisPontos()
    else:
      self.crossoverUniforme()

  def crossoverUmPonto(self):
    self.descendentes = np.empty((self.numeroDesc, self.pais.shape[1]))
    i = 0
    while i < self.numeroPais:
      ponto = rd.randint(1, self.pais.shape[1] - 1) 
      pai1Index = i%self.pais.shape[0]
      pai2Index = (i+1)%self.pais.shape[0]

      self.descendentes[i,0:ponto] = self.pais[pai1Index,0:ponto]
      self.descendentes[i,ponto:] = self.pais[pai2Index,ponto:]

      self.descendentes[i+1,0:ponto] = self.pais[pai2Index,0:ponto]
      self.descendentes[i+1,ponto:] = self.pais[pai1Index,ponto:]

      i= i+2

  def crossoverDoisPontos(self):
    self.descendentes = np.empty((self.numeroDesc, self.pais.shape[1]))
    i = 0
    while i < self.numeroPais:
      ponto1 = rd.randint(1, self.pais.shape[1] - 3) 
      ponto2 = rd.randint(ponto1 + 1, self.pais.shape[1] - 1) 

      pai1Index = i%self.pais.shape[0]
      pai2Index = (i+1)%self.pais.shape[0]

      pai1Corte1 = self.pais[pai1Index,0:ponto1]
      pai1Meio = self.pais[pai1Index,ponto1:ponto2]
      pai1Corte2 = self.pais[pai1Index,ponto2:]

      pai2Corte1 = self.pais[pai2Index,0:ponto1]
      pai2Meio = self.pais[pai2Index,ponto1:ponto2]
      pai2Corte2 = self.pais[pai2Index,ponto2:]

      self.descendentes[i,0:ponto1] = pai1Corte1
      self.descendentes[i,ponto1:ponto2] = pai2Meio
      self.descendentes[i,ponto2:] = pai1Corte2

      self.descendentes[i+1,0:ponto1] = pai2Corte1
      self.descendentes[i+1,ponto1:ponto2] = pai1Meio
      self.descendentes[i+1,ponto2:] = pai2Corte2

      i= i+2

  def crossoverUniforme(self):
    self.descendentes = np.empty((self.numeroDesc, self.pais.shape[1]))

    i = 0
    while i < self.numeroPais:
      mascara = np.random.randint(2, size=(self.pais.shape[1]))

      pai1Index = i%self.pais.shape[0]
      pai2Index = (i+1)%self.pais.shape[0]

      j = 0
      for m in mascara:
        if m == 1:
          self.descendentes[i,j] = self.pais[pai1Index,j]
          self.descendentes[i+1,j] = self.pais[pai2Index,j]
        else:
           self.descendentes[i,j] = self.pais[pai2Index,j]
           self.descendentes[i+1,j] = self.pais[pai1Index,j]
        j = j+1
      i= i+2

  def mutacao(self):
    self.mutacoes = np.empty((self.descendentes.shape))

    #taxa de mutação
    taxaMutacao = 0.4

    for i in range(self.mutacoes.shape[0]):
      random_value = rd.random()
      self.mutacoes[i,:] = self.descendentes[i,:]
      if random_value > taxaMutacao:
        continue
      int_random_value = randint(0,self.descendentes.shape[1]-1)    
      if self.mutacoes[i,int_random_value] == 0 :
        self.mutacoes[i,int_random_value] = 1
      else :
        self.mutacoes[i,int_random_value] = 0

  def selecaoSobreviventes(self):
    aptidao = self.calculaAptidao(self.populacaoTotal)
    aptidaoTotal = np.sum(aptidao) 

    if aptidaoTotal == 0:
      self.selecaoSobreviventesAleatoria()
      return

    if self.tipoSelecaoSobreviventes == 'Elitismo':
      self.selecaoSobreviventesElitismo()
    elif self.tipoSelecaoSobreviventes == 'Ranking':
      self.selecaoSobreviventesRanking()
    else:
      self.selecaoSobreviventesAleatoria()

  def selecaoSobreviventesElitismo(self):
    aptidao = self.calculaAptidao(self.populacaoTotal)
    aptidao = list(aptidao)
    for i in range(self.populacao.shape[0]):
      #seleciona o maior fitness da lista
      aptidaoMaxima = np.where(aptidao == np.max(aptidao)) 
      #seleciona o elemento com o maior fitness da lista
      self.populacao[i,:] = self.populacaoTotal[aptidaoMaxima[0][0], :]
      #seta o maior fitness para um valor mínimo, para que o mesmo não seja selecionado novamente
      aptidao[aptidaoMaxima[0][0]] = -999999

  def selecaoSobreviventesRanking(self):
    #i    -> individuo
    #t    -> geração
    #min  -> pior fit
    #max  -> melhor fit
    #n    -> numero de individuos da população
    #E(i,t) = min + (max - min) * ((rank(i,t) - 1)/(n-1))

    aptidaoRank = self.calculaAptidao(self.populacaoTotal)
    aptidaoRank = sorted(range(len(aptidaoRank)),key=aptidaoRank.__getitem__, reverse=True)
    min = aptidaoRank[aptidaoRank[-1]]
    max = aptidaoRank[aptidaoRank[0]]
    n = self.populacaoTotal.shape[0]

    E = np.empty(self.populacaoTotal.shape[0])
    for i in range(self.populacaoTotal.shape[0]):
      rank = aptidaoRank.index(i) + 1; #maior rank = 1
      E[i] = min + (max - min) * ((rank - 1)/(n-1))

    E = list(E)
    for i in range(self.populacao.shape[0]):
      #seleciona o maior E da lista
      eMaxima = np.where(E == np.max(E)) 
      #seleciona o elemento com o maior E da lista
      self.populacao[i,:] = self.populacaoTotal[eMaxima[0][0], :]
      #seta o maior E para um valor mínimo, para que o mesmo não seja selecionado novamente
      E[eMaxima[0][0]] = -999999


  def selecaoSobreviventesAleatoria(self):
    idx = np.random.randint(self.populacaoTotal.shape[0], size=self.populacao.shape[0])
    self.populacao = self.populacaoTotal[idx,:]

  def imprimirMochilaSelecionada(self):
    for i in range(self.populacao.shape[0]):
      self.mochila.imprimirMochilaSelecionada(self.populacao[i], self.pesoTotal[i])
    

  def run(self):
    #self.mochila.imprimirMochila()
    #self.imprimirPopulacao('Inicial')
    ultimasAptidoes = list()
    j = 0
    for i in range(self.numeroGeracoes):
      self.aptidao = self.calculaAptidao(self.populacao)

      if j < self.itensCondicaoParada:
        ultimasAptidoes.append(self.aptidao.copy())
        j = j + 1
      elif np.sum(ultimasAptidoes[0]) > 100 and np.array_equal(ultimasAptidoes[0], ultimasAptidoes[self.itensCondicaoParada - 1]):
        # print('Qntd Gerações: {}'.format(i))
        # print('Condição: {}'.format(self.itensCondicaoParada))
        # print('Tamanho pop: {}'.format(self.tamanhoPopulacao))
        break
      else:
        ultimasAptidoes = list()
        j = 0

      self.selecao()
      self.crossover()
      self.mutacao()
      self.populacaoTotal = np.concatenate((self.populacao.astype(int), self.mutacoes.astype(int)))
      self.selecaoSobreviventes()
        
    #self.imprimirPopulacao('Final') 
    self.aptidao = self.calculaAptidao(self.populacao)      
    #self.imprimirAptidao('Final')
    #self.imprimirMelhorAptidao('Final')
    return self.getMelhorAptidao()
    #print("Peso Limite: {}".format(self.mochila.pesoLimite))
    #self.imprimirPesos('Final')
    #self.imprimirMochilaSelecionada()
 
  


In [ ]:
tipoSelecao = ['Roleta Viciada', 'Torneio', 'Aleatória']

tipoCrossover = ['Um Ponto de Corte', 'Dois Pontos de Corte', 'Uniforme']

tipoSelecaoSobreviventes = ['Elitismo', 'Ranking', 'Aleatória']

In [ ]:
import csv

class csvResultado:  
    def __init__(self, nomeArquivo, i, aptRoleta, tempRoleta, aptTorneio, tempTorneio, aptAleatoria, tempAleatoria):  
        self.nomeArquivo = nomeArquivo  
        self.index = i
        self.aptRoleta = aptRoleta
        self.tempRoleta = tempRoleta
        self.aptTorneio = aptTorneio
        self.tempTorneio = tempTorneio
        self.aptAleatoria = aptAleatoria
        self.tempAleatoria = tempAleatoria

def escreveCsvResultado(listaResultadosCsv, nomeArquivo):
  nomeResultadoArquivo = 'resultadoProblemaMochila' + nomeArquivo + '.csv'
  with open(nomeResultadoArquivo, 'w', newline='') as file:
    fieldnames = ['nomeArquivo', 'index', 'aptRoleta', 'tempRoleta', 'aptTorneio', 'tempTorneio', 'aptAleatoria', 'tempAleatoria']
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    for objeto in listaResultadosCsv:
      writer.writerow({'nomeArquivo': objeto.nomeArquivo, 'index': objeto.index, 'aptRoleta': objeto.aptRoleta, 'tempRoleta': objeto.tempRoleta, 'aptTorneio': objeto.aptTorneio, 'tempTorneio': objeto.tempTorneio, 'aptAleatoria': objeto.aptAleatoria, 'tempAleatoria': objeto.tempAleatoria})

In [ ]:
def runAgsForFile(nomeArquivo):
  #print('Arquivo {0} \n'.format(nomeArquivo))

  listaResultadosCsv = []

  for i in range(10):
    mochila = Mochila(nomeArquivo)
    algoritmoGeneticoInicial = InicializaAlgoritmoGenetico(mochila)
    
    #print('{0} - Seleção: {1}, Crossover: {2}, Sobreviventes: {3}'.format(i, tipoSelecao[0], tipoCrossover[0], tipoSelecaoSobreviventes[0]))
    start_time = time.time()
    agRoleta = AlgoritmoGenetico(algoritmoGeneticoInicial, tipoSelecao[0], tipoCrossover[0], tipoSelecaoSobreviventes[0])
    aptRoleta = agRoleta.run()
    tempRoleta = "{:.3f}".format(time.time() - start_time)  


    #print('{0} - Seleção: {1}, Crossover: {2}, Sobreviventes: {3}'.format(i, tipoSelecao[1], tipoCrossover[0], tipoSelecaoSobreviventes[0]))
    start_time = time.time()
    agTorneio = AlgoritmoGenetico(algoritmoGeneticoInicial, tipoSelecao[1], tipoCrossover[0], tipoSelecaoSobreviventes[0])
    aptTorneio = agTorneio.run()
    tempTorneio = "{:.3f}".format(time.time() - start_time)

    #print('{0} - Seleção: {1}, Crossover: {2}, Sobreviventes: {3}'.format(i, tipoSelecao[2], tipoCrossover[0], tipoSelecaoSobreviventes[0]))
    start_time = time.time()
    agAleatoria = AlgoritmoGenetico(algoritmoGeneticoInicial, tipoSelecao[2], tipoCrossover[0], tipoSelecaoSobreviventes[0])
    aptAleatoria = agAleatoria.run()
    tempAleatoria = "{:.3f}".format(time.time() - start_time)

    listaResultadosCsv.append(csvResultado(nomeArquivo, i, aptRoleta, tempRoleta, aptTorneio, tempTorneio, aptAleatoria, tempAleatoria))

  escreveCsvResultado(listaResultadosCsv, nomeArquivo)



In [ ]:
runAgsForFile('ks_i0012.txt')

In [ ]:
runAgsForFile('ks_i0032.txt')

In [ ]:
runAgsForFile('ks_i0100.txt')

In [ ]:
runAgsForFile('ks_i0322.txt')

In [ ]:
runAgsForFile('ks_i0450.txt')

In [ ]:
runAgsForFile('ks_i0890.txt')

In [ ]:
runAgsForFile('ks_i1200.txt')

In [ ]:
runAgsForFile('ks_i8000.txt')